## **Project 5  Car_Price_Prediction**             
        
***Выполнила Хуснуллина Наиля***       
***

# 1. Импорт библиотек

In [32]:
import numpy as np 
import pandas as pd 
import re

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

In [33]:
# зафиксируем версию пакетов
!pip freeze > requirements.txt

In [34]:
# фиксируем RANDOM_SEED
RANDOM_SEED = 42

# 2. Импорт данных

In [35]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

PATH_to_file = '/kaggle/input/sf-dst-car-price-prediction/'
PATH_to_file_data = '/kaggle/input/auto-ru-parsing-06-01-2021/'
PATH_to_file_fit_data = '/kaggle/input/featured-df/'

# Импорт обработанного датасета

In [36]:
#импорт для каггла
#df = pd.read_csv(PATH_to_file_fit_data +'auto_featured.csv') 

#submit = pd.read_csv(PATH_to_file+'sample_submission.csv')

#импорт для локального
df = pd.read_csv('auto_featured.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submission.csv')

## Подготовка к ML

In [37]:
# Возвращаем первоначальное разбиение на train и test

X_sub_price = df.query('Train == 0').drop(['Train'], axis=1)
X_sub = X_sub_price.drop(['price_RUR'], axis=1).values

train_data = df[df['Train'] == 1].drop(['Train'], axis=1)
test_data = df[df['Train'] == 0].drop(['Train'], axis=1)

y = train_data.price_RUR.values           
X = train_data.drop(['price_RUR'], axis=1).values

### train_test_split 

In [38]:
# Из train выделяем данные на валидацию 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((34686, 32), (74009, 32), (74009, 31), (59207, 31), (14802, 31))

### Метрики качества

In [39]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# МОДЕЛЬ 1           
### Случайный лес

In [40]:
model_1 = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model_1.fit(X_train, np.log(y_train+1))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   18.3s finished


RandomForestRegressor(n_jobs=-1, random_state=42, verbose=1)

In [41]:
predict_test = np.exp(model_1.predict(X_test))
predict_submission = np.exp(model_1.predict(X_sub))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [42]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.64%


In [43]:
submit['price'] = predict_submission
#submit.to_csv('submission_2_v1.csv', index=False)
submit.head(10)

,sell_id,price
0,1100575026,6.960671e+05
1,1100549428,1.022369e+06
2,1100658222,8.783287e+05
3,1100937408,8.176029e+05
4,1101037972,7.855634e+05
5,1100912634,7.558236e+05
6,1101228730,6.952376e+05
7,1100165896,4.418092e+05
8,1100768262,1.949931e+06
9,1101218501,7.682438e+05


***Kaggle 14.58***

# Модель 2      
### GradientBoostingRegressor

In [44]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=1000)
gb.fit(X_train, np.log(y_train+1))

GradientBoostingRegressor(learning_rate=0.03, max_depth=10, n_estimators=1000)

In [45]:
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))

In [46]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.44%


In [47]:
submit['price'] = predict_submission
#submit.to_csv('submission_2_v2.csv', index=False)
submit.head(10)

,sell_id,price
0,1100575026,6.525466e+05
1,1100549428,1.013686e+06
2,1100658222,9.430875e+05
3,1100937408,7.986098e+05
4,1101037972,8.065482e+05
5,1100912634,7.975768e+05
6,1101228730,6.739610e+05
7,1100165896,4.273512e+05
8,1100768262,1.972047e+06
9,1101218501,7.366566e+05


***Kaggle 14.22***

# Модель 3      
### Стекинг

In [48]:
X_pred = X_sub

In [49]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_pred = scaler.transform(X_pred)

y_train = y_train
y_test = y_test

In [50]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def compute_metric(regr, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    return np.round(mape(y_test, y_test_pred)*100, 4)

In [51]:
def compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred):    
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
    
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)
    
    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(X_pred)
    
    return X_meta_train, X_meta_test, X_meta_pred

In [52]:
def generate_meta_features2(regr_s, X_train, X_test, y_train, cv, X_pred):
   
    features = [compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred) for regr in tqdm(regr_s)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

In [53]:
stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features2([
    RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                          max_features=7,max_depth=77, bootstrap=True, random_state=42),
    AdaBoostRegressor(random_state=42),
    ExtraTreesRegressor(random_state=42),
    RandomForestRegressor(random_state=42)], X_train, X_test, y_train, cv, X_pred)

regr = RandomForestRegressor(
 n_estimators=300,
 min_samples_split=2,
 min_samples_leaf=1,
 max_features=3,
 max_depth=77,
 bootstrap=True, 
 random_state=42)

print(f"Точность модели по метрике MAPE: {compute_metric(regr, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test, y_test=y_test)}%")

100%|██████████| 4/4 [12:25<00:00, 186.50s/it]


Точность модели по метрике MAPE: 11.8306%


In [54]:
predict_submission = regr.predict(stacked_features_pred)

In [55]:
submit['price'] = predict_submission
#submit.to_csv('submission_2_v3.csv', index=False)
submit.head(10)

,sell_id,price
0,1100575026,6.265267e+05
1,1100549428,1.021097e+06
2,1100658222,8.700217e+05
3,1100937408,8.353123e+05
4,1101037972,8.214093e+05
5,1100912634,7.097973e+05
6,1101228730,6.995193e+05
7,1100165896,4.156867e+05
8,1100768262,1.924411e+06
9,1101218501,8.485824e+05


***Kaggle 16.38 ***

# Модель 4      
### Xgboost

In [56]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [57]:
import xgboost as xgb

In [58]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate=0.05, max_depth=12, alpha=1,
                          n_estimators=1000)
xg_reg.fit(X_train, y_train)
xg_red_pred = xg_reg.predict(X_test)

In [59]:
print(f"Точность модели по метрике MAPE: {(mape(np.exp(y_test), np.exp(xg_red_pred)))*100:0.2f}%")

Точность модели по метрике MAPE: 10.09%


In [60]:
predict_submission = np.exp(xg_reg.predict(X_sub))

In [61]:
submit['price'] = predict_submission
#submit.to_csv('submission_2_v4.csv', index=False)
submit.head(10)

,sell_id,price
0,1100575026,5.977410e+05
1,1100549428,6.714241e+05
2,1100658222,6.787829e+05
3,1100937408,6.476183e+05
4,1101037972,7.300761e+05
5,1100912634,7.849638e+05
6,1101228730,7.832499e+05
7,1100165896,7.065156e+05
8,1100768262,1.018298e+06
9,1101218501,8.016116e+05


***Kaggle 13.93 ***

# Submission

submit['price'] = predict_submission
submit.to_csv('submission_2_v4.csv', index=False)
submit.head(10)

**Резюме**          

В ходе выполнения задания по проекту 5 получены навыки парсинга, собраны данные для обучающего датасета.        

Собранные данные обработаны, обучающий и тестовый датасет приведены к единому виду, получены новые признаки путем разбиения фичей регулярными выражениями, а также пуем выполнения арифметических операций над имеющимися данными.        

Построены ряд моделей для предсказания цены на авто, а именно:
    - RandomForestRegressor      
    - GradientBoostingRegressor       
    - стекинг моделей RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor           
    - Xgboost        
         
Оценка моделей проводилась минимизацией метрики МАРЕ (Mean Absolute Percentage Error).      
Наилучший результат показал Xgboost.             
            
            
Возможные пути для улучшения показателя на каггле:                    
- удаление выбросов по числовым параметрам при обработке датасета               
- подбор параметров для рассмотренных моделей              
- рассмотрение таких моделей как CatBoost, LightGBM, и тд              
- подбор моделей и параметров к ним для стекинга, блендинга             